In [ ]:
import numpy as np
import os, sys, cv2
import matplotlib.pyplot as plt
import json
sys.path.append("../../")
from opensfm import dataset
reload(dataset)

args = {}
#args["dataset"] = "../../data/nexar_dup"
args["dataset"] = "../../data/47fa6807-46b4-4121-848b-beb291cc2d60/"

data = dataset.DataSet(args["dataset"])
images = data.images()

In [ ]:
reconstructions = data.load_reconstruction()

In [ ]:
# global flags
relative_seg_path = "output/results/joint/"

assert len(reconstructions) == 1, "reconstruction falls into several parts"
# get a bunch of 3D locations indicated by those points
graph = data.load_tracks_graph()

In [ ]:
import random
import numpy as np

class plane(object):
    def add_1col(self, points):
        points = np.array(points)
        npoints = points.shape[0]
        points = np.concatenate((points, np.ones((npoints, 1))), axis=1)
        return points

    def __init__(self, points):
        points = self.add_1col(points)
        #t0 = time.time()
        corre = np.dot(points.T, points)
        W, U = np.linalg.eigh(corre)
        U = np.fliplr(U)
        #U0, s, V = np.linalg.svd(points.T)
        #global timet
        #timet += time.time()-t0 
        self.coeff = U[:, -1]
        self.coeff = np.reshape(self.coeff, (1,4))
        
    def error(self, points):
        # calculate the RMSE of these points, relative to the plane
        points = self.add_1col(points)
        residual = np.dot(self.coeff, points.T)
        return np.sqrt(np.mean(residual**2))
    
    def error_singles(self, points):
        points = self.add_1col(points)
        residual = np.dot(self.coeff, points.T)
        residual = np.reshape(residual, (-1))
        return np.abs(residual)
        
def ransac(points, maxIter, threshold, minN, goodN):
    t0 = time.time()
    bestfit = None
    besterr = 1e9
    inliers = None

    points = np.array(points)
    n = points.shape[0]
    if n <= goodN:
        print("too less points")
        return None, None
        
    for i in range(maxIter):
        maybeInliers = random.sample(range(n), minN)
        maybeModel = plane(points[maybeInliers, :])
        allInliers = maybeModel.error_singles(points) < threshold
        
        if allInliers.sum() > goodN:
            hyp_points = points[allInliers,:]
            betterModel = plane(hyp_points)
            thiserror = betterModel.error(hyp_points)
            if thiserror < besterr:
                inliers = allInliers
                besterr = thiserror
                bestfit = betterModel
    
    print("best error is %f, with %d inliers, out of %d total (%f second)" % 
          (besterr, inliers.sum(), n, time.time()-t0))
    return bestfit, inliers

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
dists = []

for shot_id in sorted(reconstructions[0].shots.keys(), reverse=True):
    t1 = time.time()
    print(shot_id)
    # 1. load the current camera center location
    shot = reconstructions[0].shots[shot_id]
    pose = shot.pose
    camera_loc = pose.get_origin()
    
    # 2. calculate the current scale
    # load all keypoints and filter out the road
    p1, f1, c1 = data.load_features(shot_id)
    ind = data.filter_by_seg(shot_id, p1, lambda x: x==0, relative_seg_path)
    if True:
        # visualize filtered feature points
        ot = np.array(p1[ind, :2])
        im = data.image_as_array(shot_id)
        plt.imshow(im)
        plt.plot(640+ot[:,0]*1280, 360+ot[:,1]*1280, 'ro')    
        plt.show()
        break
    road_keypoint_ids = np.array(range(len(ind)))[ind]
    
    # convert 2D points to 3D track ids
    track_ids, selected = dataset.observations_to_tracks(graph, shot_id, road_keypoint_ids, p1)
    if len(track_ids) < 15:
        print("too less points")
        dists.append(None)
        continue
    
    if False:
        # visualize all tracks
        ot = []
        for track, d in graph[shot_id].items():
            f=d['feature']
            ot.append([f[0], f[1]])
        ot = np.array(ot)
        im = data.image_as_array(shot_id)
        plt.imshow(im)
        plt.plot(640+ot[:,0]*1280, 360+ot[:,1]*1280, 'ro')    
        plt.show()
        
    if False:
        # visualize all selected road points
        ot = p1[selected, :2]
        im = data.image_as_array(shot_id)
        plt.imshow(im)
        plt.plot(640+ot[:,0]*1280, 360+ot[:,1]*1280, 'ro')    
        plt.show()
        break

    # map tracks to the 3D coordinates
    coordinates = []
    for tid in track_ids:
        if tid in reconstructions[0].points:
            coordinates.append(reconstructions[0].points[tid].coordinates)
    coordinates = np.array(coordinates)
    #print(coordinates)
    threshold = np.mean(np.percentile(coordinates, 75, axis=0) - np.percentile(coordinates, 25, axis=0)) / 5
    t2 = time.time()
    print("using threshold %f (%f second)" % (threshold, t2-t1))
    
    bestModel, inliers = ransac(coordinates, 200, threshold, 3, 15)
    
    if not(bestModel is None):
        coordinates = coordinates[inliers, :]

        # calculate the distance between plane and camera center
        dist = bestModel.error_singles(camera_loc.reshape((1,3)))
        print(dist)
        dists.append(dist)
    else:
        dists.append(None)
    
    if False:
        # fit a plane with a robust method
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(coordinates[:,0], coordinates[:, 1], coordinates[:, 2])
        plt.show()
    
    if shot_id == "1158.jpg":
        break

In [ ]:
a=np.array(dists)
a=a.reshape((-1))

In [ ]:
#plt.plot(np.log(a))
plt.figure()
plt.plot(a)

In [ ]:
import pickle
camera_heights = np.array(dists)
camera_heights = camera_heights.reshape((-1))
fname = os.path.join(args['dataset'], "raw_estimated_scale.pkl")
with open(fname, "w") as f:
    pickle.dump(camera_heights, f)

In [ ]:
t=pickle.load(open(fname, "r"))
plt.plot(t)

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
filtered = lowess(t, range(len(t)), is_sorted=True, frac=0.055, it=0)
plt.plot(filtered[:,0], filtered[:,1], 'b')